## Notebook to extract life expectancy data from all the IHME years

In [1]:
## importing necessary libraries

import pandas as pd
import os

In [2]:
folder_path = '../get_LE_data/Life Expectency Data'

In [3]:
# List all the csv files in the folder

csv_files = [x for x in os.listdir(folder_path) if x.endswith('.CSV')] ## this is an example of list comprehension
csv_files

['IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2000_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2001_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2002_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2003_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2004_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2005_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2006_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2007_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2008_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2009_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2010_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2011_ALL_BOTH_Y2023M06D12.CSV',
 'IHME_USA_COD_COUNTY_RACE_ETHN_2000_2019_LT_2012_ALL_BOTH_Y2023M06D12.CSV',

In [4]:
type(csv_files)

list

In [5]:
# lets create an empty list to store the dataframes
dataframes = []

# lets loop over the list of CSV files and read each one

for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dataframes.append(df)


In [6]:
print(type(dataframes))
print(len(dataframes)) ## A total of 20 dataframes, stored as a list


<class 'list'>
20


In [7]:
reduced_dataframe=[]

for i in range (0,20):
    
    ## lets extract the total life expectency and only of the age group less than 1 year olds.

    df2=dataframes[i].loc[(dataframes[i]['race_name'] == 'Total') & (dataframes[i]['age_name'] == '<1 year')]

    ## lets remove empty cells

    df3=df2.dropna()

    ## As the dataframe consists of life expectencey at the state level as well
    ## lets gather only those with county, since fips for state end at 56, will set the condition to be greater than this to get the data at the county level.

    df4=df3.loc[(df3['fips'] > 60)]


    ## lets delete these columns. 
    df5 = df4.drop(['measure_id', 'location_id','measure_name', 'race_id', 'race_name', 'sex_id', 'sex_name', 'age_group_id',
     'age_name', 'metric_id', 'metric_name', 'upper', 'lower'], axis=1)

    ## lets rename the column val to mean life expectency. 
    df5 = df5.rename(columns={'val': 'MeanLifeExpectency'})

    reduced_dataframe.append(df5)


In [8]:
print(type(reduced_dataframe))
print(len(reduced_dataframe)) 

<class 'list'>
20


In [9]:
final_df=pd.concat(reduced_dataframe,ignore_index=True)
final_df

,location_name,fips,year,MeanLifeExpectency
0,Autauga County (Alabama),1001.0,2000,74.712886
1,Baldwin County (Alabama),1003.0,2000,76.699950
2,Barbour County (Alabama),1005.0,2000,74.009403
3,Bibb County (Alabama),1007.0,2000,73.201889
4,Blount County (Alabama),1009.0,2000,75.274624
...,...,...,...,...
62535,Sweetwater County (Wyoming),56037.0,2019,78.018892
62536,Teton County (Wyoming),56039.0,2019,85.638149
62537,Uinta County (Wyoming),56041.0,2019,77.521691
62538,Washakie County (Wyoming),56043.0,2019,78.137158


In [10]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62540 entries, 0 to 62539
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   location_name       62540 non-null  object 
 1   fips                62540 non-null  float64
 2   year                62540 non-null  int64  
 3   MeanLifeExpectency  62540 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 1.9+ MB


In [11]:
final_df.describe()

,fips,year,MeanLifeExpectency
count,62540.000000,62540.000000,62540.000000
mean,30297.793092,2009.500000,77.030626
std,15248.203271,5.766327,2.462231
min,1001.000000,2000.000000,65.176273
25%,18157.000000,2004.750000,75.351288
50%,29149.000000,2009.500000,77.133494
75%,45083.000000,2014.250000,78.709520
max,56045.000000,2019.000000,92.253858


In [12]:
final_df.to_csv("All_year_dataframe.csv")